# Reinforcement Learning, 2024-01 
# Tarea 2 - Programación Dinámica

> Daniel Villar González, 201923374.  
> Daniel Alvarez, 201911320.

**Considere la variación del juego escaleras y serpientes mostrada en la figura:**
<center><img src="imagen-resource-h2.PNG" alt="drawing" width="400"/>
</center> 

- **La meta del jugador es ganar la partida llegando a una de las casillas marcadas en azul.**
- **El jugador pierde la partida si cae en una de las casillas marcadas en rojo.**
- **En cada jugada, antes de lanzar el dado, el jugador decide si quiere avanzar o retroceder el número de casillas indicadas por el dado.**
- **El dado está cargado, con probabilidades p =[p1 p2 p3 p4 p5 p6].**
- **En las casillas 1 y 100 la ficha rebota (si se supera el extremo, se avanza en la otra dirección la cantidad restante).**


## **Librerias**

In [1]:
import numpy as np

## **Modelado de ambiente**

**Modele el esquema de juego tal que cumpla con las condiciones establecidas para el agente en el ambiente**

In [2]:
class Snakeladders:
    def __init__(self, p_dice, blue_coordinates, red_coordinates):
        self.tablero = np.arange(1, 101)
        self.blue_coordinates = blue_coordinates
        self.red_coordinates = red_coordinates
        self.p_dice= p_dice
        self.actual_pos = 1
    
    def launch_dice(self):
        return np.random.choice(np.arange(1, 7), p=self.p_dice)
    
    def bounce(self, new_position):
        if new_position < 1:
            return 1 - new_position
        elif new_position > 100:
            return 200 - new_position
        else:
            return new_position
    
    def move_token(self, dice):
        new_position = self.actual_pos + dice
        new_position = self.bounce(new_position)
        if new_position in self.blue_coordinates:
            print("BLUE BOX!!! WINNER")
            self.posicion_actual = new_position
            return True
        elif new_position in self.red_coordinates:
            print("RED BOX!! LOSSER :(")
            self.posicion_actual = 0
            return False
        else:
            self.actual_pos = new_position
            return True
    
    def play_game(self):
        while self.actual_pos > 0 and self.actual_pos <= 100:
            print("Your box is: ", self.actual_pos)
            decision = input("Do you want to advance (A) or return (R)? ")
            if decision.upper() == 'A':
                dado = self.launch_dice()
                print("Your dice is :", dado)
            elif decision.upper() == 'R':
                dado = -self.launch_dice()
                print("Your dice is:", abs(dado))
            elif decision.upper() == 'C':
                break
            else:
                print("Invalid decision!!")
                continue
            if not self.move_token(dado):
                print("End Game.")
                break


p_dice = [0.1, 0.1, 0.2, 0.2, 0.2, 0.2]

blue_coordinates = [100, 80]
red_coordinates = [21, 37, 45, 65, 89]

game = Snakeladders(p_dice, blue_coordinates, red_coordinates)
game.play_game()


Your box is:  1


## **Punto 1**

**Modele este problema como un MDP. De detalladamente todos los elementos del MDP: estados, recompensas, acciones y p(s′, r | s, a) ∀s, s′, r, a, y factor de descuento γ.**

Estados (S): Los estados en este problema representarán la posición actual del jugador en el tablero. Cada casilla en el tablero será un estado posible, desde la casilla 1 hasta la casilla 100: S = {1, 2, 3, ..., 99, 100}.

Recompensas (R): Se asigna una recompensa positiva (+1) cuando el jugador llega a una casilla marcada en azul (la meta) y una recompensa negativa (-1) cuando el jugador cae en una casilla marcada en rojo (pierde la partida). En el resto de las casillas la recompensa será 0. 

Acciones (A): En cada estado, el jugador puede tomar una acción que consiste en decidir si avanza o retrocede el número de casillas indicadas por el dado. Dado que el dado está cargado, el jugador también debe considerar la probabilidad de avanzar o retroceder en función de los resultados posibles del dado: A = {Avanzar, Retroceder}

Probabilidad de transición (p(s', r | s, a)): Define la probabilidad de pasar de un estado s a un estado s′ y recibir una recompensa r dado que se toma una acción a. En este caso, la probabilidad de transición depende de la acción tomada (avanzar o retroceder) y del resultado del dado. Más adelante se definirá la matriz de probabilidades de transición.

Factor de descuento (γ): Se utilizará un valor de descuento γ=0.9 para dar más peso a las recompensas inmediatas.

A continuación se definirán las probabilidades de transición para cada posible estado:

In [3]:
p1 = 1
p2 = 2
p3 = 3
p4 = 4
p5 = 5
p6 = 6
p = [p1, p2, p3, p4, p5, p6]
transiciones = {8:26, 21:82, 43:77, 50:91, 54:93, 62:96, 66:87, 80:100, 98:28, 95:24, 92:51, 83:19, 73:1, 69:33, 64:36, 59:17, 55:7, 52:11, 48:9, 46:5, 44:22}
recompensas = {23:-1, 37:-1, 45:-1, 67:-1, 89:-1, 80:1, 100:1}
    
matrizAdelante = []
for i in range(100):
    matrizAdelante.append([0] * 100)

matrizRecompensasAdelante = []
for i in range(100):
    matrizRecompensasAdelante.append([0] * 100)

llaves_transiciones = transiciones.keys()
llaves_recompensas = recompensas.keys()

for i in range(len(matrizAdelante)): 
    for j in range(0, 6):
        destino = i + j + 1 
        if destino >= 100:  
            destino = 99 - (destino - 100)  
        if destino+1 in llaves_transiciones:
            matrizAdelante[i][transiciones[destino+1]-1] = p[j]  
            if destino+1 in llaves_recompensas:
                matrizRecompensasAdelante[i][transiciones[destino+1]-1] = recompensas[destino+1]
        else:
            matrizAdelante[i][destino] = p[j] 
            if destino+1 in llaves_recompensas:
                matrizRecompensasAdelante[i][destino] = recompensas[destino+1]
        
for i in range(len(matrizAdelante)):
    print(matrizAdelante[i])

[0, 1, 2, 3, 4, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 2, 3, 4, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 2, 3, 4, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 2, 3, 0, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 

In [4]:
transpuesta = []
for j in range(len(matrizAdelante[0])):
    fila_transpuesta = []
    for i in range(len(matrizAdelante)):
        fila_transpuesta.append(matrizAdelante[i][j])
    transpuesta.append(fila_transpuesta)

Como se puede notar, quedaron definidas las probabilidades de transición desde un s a un estado s′ dado que se toma la acción de ir hacia adelante. Las filas indican el estado s y las columnas el estado s'. El valor de la matriz en la posición (s,s') indica la probabilidad de pasar desde el estado s al estado s', las cuales dependen de los valores de **p**. A continuación se muestra la recompensa obtenida al tomar la acción de ir hacia adelante, desde un estado s:

In [5]:
for i in range(len(matrizRecompensasAdelante)):
    print(matrizRecompensasAdelante[i])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 


A continuación se definirán las probabilidades de transición para la acción de ir hacia atrás:

In [6]:
matrizAtras = []
for i in range(100):
    matrizAtras.append([0] * 100)

matrizRecompensasAtras= []
for i in range(100):
    matrizRecompensasAtras.append([0] * 100)

for i in range(len(matrizAtras)): 
    for j in range(0, 6):
        destino = i - j 
        if destino <= 0:  
            destino = abs(destino) + 1
        if destino in llaves_transiciones:
            matrizAtras[i][transiciones[destino]-1] = p[j]  
            if destino in llaves_recompensas:
                matrizRecompensasAtras[i][transiciones[destino]-1] = recompensas[destino]
        else:
            matrizAtras[i][destino-1] = p[j] 
            if destino in llaves_recompensas:
                matrizRecompensasAtras[i][destino-1] = recompensas[destino]

for i in range(len(matrizAtras)):
    print(matrizAtras[i])

[1, 2, 3, 4, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[2, 3, 4, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[3, 4, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[4, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Como se puede notar, quedaron definidas las probabilidades de transición desde un s a un estado s′ dado que se toma la acción de ir hacia atrás.

In [7]:
for i in range(len(matrizRecompensasAtras)):
    print(matrizRecompensasAtras[i])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Como se puede notar, quedaron definidas las recompensas obtenidas desde un s a un estado s′ dado que se toma la acción de ir hacia atrás.

De esta forma, quedan definidas las probabilidades de transición p(s′, r | s, a) ∀s, s′, r, a.

In [8]:
class Snakeladders:
    def __init__(self, p_dice, blue_coordinates, red_coordinates):
        self.tablero = np.arange(1, 101)
        self.blue_coordinates = blue_coordinates
        self.red_coordinates = red_coordinates
        self.p_dice= p_dice
        self.actual_pos = 1
        self.state_space = {i: 0 for i in range(1, 101)}

        for pos in blue_coordinates:
            self.state_space[pos] = 1
        for pos in red_coordinates:
            self.state_space[pos] = -1

    def launch_dice(self):
        return np.random.choice(np.arange(1, 7), p=self.p_dice)
    
    def bounce(self, new_position):
        if new_position < 1:
            return 1 - new_position
        elif new_position > 100:
            return 200 - new_position
        else:
            return new_position
    
    def move_token(self, dice):
        new_position = self.actual_pos + dice
        new_position = self.bounce(new_position)
        if new_position in self.blue_coordinates:
            print("BLUE BOX!!! WINNER")
            self.posicion_actual = new_position
            return True
        elif new_position in self.red_coordinates:
            print("RED BOX!! LOSSER :(")
            self.posicion_actual = 0
            return False
        else:
            self.actual_pos = new_position
            return True
    
    def play_game(self):
        while self.actual_pos > 0 and self.actual_pos <= 100:
            print("Your box is: ", self.actual_pos)
            decision = input("Do you want to advance (A) or return (R)? ")
            if decision.upper() == 'A':
                dado = self.launch_dice()
                print("Your dice is :", dado)
            elif decision.upper() == 'R':
                dado = -self.launch_dice()
                print("Your dice is:", abs(dado))
            elif decision.upper() == 'C':
                break
            else:
                print("Invalid decision!!")
                continue
            if not self.move_token(dado):
                print("End Game.")
                break


p_dice = [0.1, 0.1, 0.2, 0.2, 0.2, 0.2]

blue_coordinates = [100, 80]
red_coordinates = [21, 37, 45, 65, 89]

game = Snakeladders(p_dice, blue_coordinates, red_coordinates)
game.play_game()


Your box is:  1


## **Punto 2**

**Escriba un módulo de Python para el MDP formulado. Su implementación debe tener métodos para:**

- **Construir el MDP para una localización dada de las casillas azules y rojas, el vector p, y el factor de descuento γ.**
- **Dada una política π (estocástica o determinística), calcular la función de valor de estado usando programación dinámica.**
- **Dada una política π (estocástica o determinística), determinar si esta política es óptima chequeando las ecuaciones de optimalidad de Bellman.**
- **Hallar una política óptima para una instancia del MDP, usando iteración de política.**
- **Hallar una política óptima para una instancia del MDP, usando iteración de valor.**


### **Construir el MDP para una localización dada de las casillas azules y rojas, el vector p, y el factor de descuento γ.**

A continuación se define un código para construir el MDP dadas las casillas azules, las casillas rojas, el vector **p** y el factor de descuento γ.

El input de las casillas azules (casillasAzules) sería una lista con los números de las casillas.
El input de las casillas rojas (casillasRojas) sería una lista con los números de las casillas.
El input del vector **p** es una lista de 6 posiciones con las probabilidades p1, p2, p3, p4, p5 y p6.
El input del factor de descuento γ (factorDescuento) es un valor entre 0 y 1. 

In [9]:
def construirMDP(casillasAzules, casillasRojas, p, factorDescuento):
    if factorDescuento > 1 or factorDescuento < 0:
        return "El valor de descuento no está entre los límites permitidos"
    matrizAdelante = []
    for i in range(100):
        matrizAdelante.append([0] * 100)
    matrizRecompensasAdelante = []
    for i in range(100):
        matrizRecompensasAdelante.append([0] * 100)
    matrizAtras = []
    for i in range(100):
        matrizAtras.append([0] * 100)
    matrizRecompensasAtras= []
    for i in range(100):
        matrizRecompensasAtras.append([0] * 100)
    recompensas = {}
    for i in range(len(casillasAzules)):
        recompensas[casillasAzules[i]] = 1
    for i in range(len(casillasRojas)):
        recompensas[casillasRojas[i]] = -1
    transiciones = {8:26, 21:82, 43:77, 50:91, 54:93, 62:96, 66:87, 80:100, 98:28, 95:24, 92:51, 83:19, 73:1, 69:33, 64:36, 59:17, 55:7, 52:11, 48:9, 46:5, 44:22}
    llaves_transiciones = transiciones.keys()
    llaves_recompensas = recompensas.keys()
    for i in range(len(matrizAdelante)): 
        for j in range(0, 6):
            destino = i + j + 1 
            if destino >= 100:  
                destino -= 100  
            if destino+1 in llaves_transiciones:
                matrizAdelante[i][transiciones[destino+1]-1] = p[j]  
                if destino+1 in llaves_recompensas:
                    matrizRecompensasAdelante[i][transiciones[destino+1]-1] = recompensas[destino+1]
            else:
                matrizAdelante[i][destino] = p[j] 
                if destino+1 in llaves_recompensas:
                    matrizRecompensasAdelante[i][destino] = recompensas[destino+1]
    for i in range(len(matrizAtras)): 
        for j in range(0, 6):
            destino = i - j 
            if destino <= 0:  
                destino += 100
            if destino in llaves_transiciones:
                matrizAtras[i][transiciones[destino]-1] = p[j]  
                if destino in llaves_recompensas:
                    matrizRecompensasAtras[i][transiciones[destino]-1] = recompensas[destino]
            else:
                matrizAtras[i][destino-1] = p[j] 
                if destino in llaves_recompensas:
                    matrizRecompensasAtras[i][destino-1] = recompensas[destino]
    return matrizAdelante, matrizAtras, matrizRecompensasAdelante, matrizRecompensasAtras

### INPUTS
casillasAzules = [80, 100]    
casillasRojas = [23, 37, 45, 67, 89]
p = [1/6, 1/6, 1/6, 1/6, 1/6, 1/6]
factorDescuento = 0.9
matrizAdelante, matrizAtras, matrizRecompensasAdelante, matrizRecompensasAtras = construirMDP(casillasAzules, casillasRojas, p, factorDescuento)

A continuación se muestran los resultados obtenidos, es decir, (1) la matriz de probabilidades de transición al tomar la acción de ir hacia adelante, (2) la matriz de recompensas obtenidas al ir hacia adelante, (3) la matriz de probabilidades de transición al tomar la acción de ir hacia atrás y (4) la matriz de recompensas obtenidas al ir hacia atrás.

In [10]:
for i in range(len(matrizAdelante)):
    print(matrizAdelante[i])

[0, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.16666666666666666, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0, 0.16666666666666666, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.16666666666666

In [11]:
for i in range(len(matrizRecompensasAdelante)):
    print(matrizRecompensasAdelante[i])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [12]:
for i in range(len(matrizAtras)):
    print(matrizAtras[i])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.16666666666666666, 0, 0, 0, 0.16666666666666666, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.16666666666666666, 0.16666666666666666, 0, 0.16666666666666666, 0.16666666666666666]
[0.16666666666666666, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.16666666666666666, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.16666666666666666, 0.16666666666666666, 0, 0.16666666666666666, 0.16666666666666666]
[0.16666666666666666, 0.16666666666666666, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.16666666666666666, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [13]:
for i in range(len(matrizRecompensasAtras)):
    print(matrizRecompensasAtras[i])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

### **Dada una política π (estocástica o determinística), calcular la función de valor de estado usando programación dinámica.**

In [14]:
politica = []
for i in range(100):
    politica.append([0.5, 0.5])

In [15]:
def calcularFuncionValorEstado(matrizAdelante, matrizRecompensasAdelante, matrizAtras, matrizRecompensasAtras, politica, factorDescuento, epsilon):
    valores = [0] * len(matrizAdelante)
    convergencia = False
    while not convergencia: 
        valores_previos = valores.copy()
        for estado in range(len(matrizAdelante)):
            nuevo_valor = 0
            # Avanzar
            for estado_siguiente in range(len(matrizAdelante[estado])):
                nuevo_valor += politica[estado][0] * matrizAdelante[estado][estado_siguiente] * (matrizRecompensasAdelante[estado][estado_siguiente] + factorDescuento*valores[estado_siguiente])
            # Retroceder
            for estado_siguiente in range(len(matrizAtras[estado])):
                nuevo_valor += politica[estado][1] * matrizAtras[estado][estado_siguiente] * (matrizRecompensasAtras[estado][estado_siguiente] + factorDescuento*valores[estado_siguiente])
            valores[estado] = nuevo_valor
        # Verificar si se ha alcanzado la convergencia
        dif_maxima = max(abs(valores[estado] - valores_previos[estado]) for estado in range(len(matrizAdelante))) 
        if dif_maxima < epsilon:
            convergencia = True
    return valores

epsilon = 0.001
V = calcularFuncionValorEstado(matrizAdelante, matrizRecompensasAdelante, matrizAtras, matrizRecompensasAtras, politica, factorDescuento, epsilon)
print("La política es: " + str(politica))
print("El valor de descuento es: " + str(factorDescuento))
print("Las probabilidades del dado son: " + str(p))
print("Función de Valor de Estado:")
for i in range(len(V)):
    print("Casilla " +str(i+1)+" tiene un valor de "+ str(V[i]))

La política es: [[0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5],

### **Dada una política π (estocástica o determinística), determinar si esta política es óptima chequeando las ecuaciones de optimalidad de Bellman.**

Para este caso se hara uso de de las ecuaciones de optimalidad de Bellman en particular la que nos da el valor para la política $v_{*}(s)$, dada por la siguiente expresión:
$$v_{*}(s)= max_{x\in A(s)} q_{\pi_{*}}(s,a)$$
Donde es más fácil verla como 
$$ max_{a}\sum_{s',r}p(s',r|s,a)[r + \lambda v_{*}(s')]$$
Para este caso se definió un vector de valores en los cuales se aplica una suma sobre estados y recompensas comparando ambos estados.

In [29]:
def bellman_cal(matrizAdelante, matrizRecompensasAdelante, matrizAtras, matrizRecompensasAtras, factorDescuento, acciones):
    valores = [0] * len(matrizAdelante)
    for estado in range(len(matrizAdelante)):
        nuevo_valor = [0] * acciones
        for estado_siguiente in range(len(matrizAdelante[estado])):
            nuevo_valor[0] += matrizAdelante[estado][estado_siguiente] * (matrizRecompensasAdelante[estado][estado_siguiente] + factorDescuento*valores[estado_siguiente])
        for estado_siguiente in range(len(matrizAtras[estado])):
            nuevo_valor[1] += matrizAtras[estado][estado_siguiente] * (matrizRecompensasAtras[estado][estado_siguiente] + factorDescuento*valores[estado_siguiente])
        valores[estado] = max(nuevo_valor)
    return valores
acciones = 2
Bellman = bellman_cal(matrizAdelante, matrizRecompensasAdelante, matrizAtras, matrizRecompensasAtras, factorDescuento, acciones)

In [56]:
print(Bellman[0])

0.16666666666666666


### **Hallar una política óptima para una instancia del MDP, usando iteración de política**

In [31]:
def mejorarPolitica(matrizAdelante, matrizRecompensasAdelante, matrizAtras, matrizRecompensasAtras, valores, factorDescuento):
    politica_nueva = [[0, 0] for _ in range(len(matrizAdelante))]
    politica_estable = True
    for estado in range(len(matrizAdelante)):
        acciones_valor = []
        valor_avanzar = sum(matrizAdelante[estado][estado_siguiente] * (matrizRecompensasAdelante[estado][estado_siguiente] + factorDescuento * valores[estado_siguiente]) for estado_siguiente in range(len(matrizAdelante[estado])))
        acciones_valor.append(valor_avanzar)
        valor_retroceder = sum(matrizAtras[estado][estado_siguiente] * (matrizRecompensasAtras[estado][estado_siguiente] + factorDescuento * valores[estado_siguiente]) for estado_siguiente in range(len(matrizAtras[estado])))
        acciones_valor.append(valor_retroceder)
        mejor_accion = acciones_valor.index(max(acciones_valor))
        if politica_nueva[estado][mejor_accion] != 1:
            politica_estable = False
        politica_nueva[estado] = [0, 0]
        politica_nueva[estado][mejor_accion] = 1
    return politica_nueva, politica_estable

In [19]:
def iteracionPolitica(matrizAdelante, matrizRecompensasAdelante, matrizAtras, matrizRecompensasAtras, factorDescuento, epsilon, error):
    politica = [[0.5, 0.5] for _ in range(len(matrizAdelante))]
    politica_estable = False
    V = [0] * len(matrizAdelante)
    while not politica_estable:
        V_anterior = V.copy()
        V = calcularFuncionValorEstado(matrizAdelante, matrizRecompensasAdelante, matrizAtras, matrizRecompensasAtras, politica, factorDescuento, epsilon)
        error_maximo = max(abs(v - v_anterior) for v, v_anterior in zip(V, V_anterior))
        politica, politica_estable = mejorarPolitica(matrizAdelante, matrizRecompensasAdelante, matrizAtras, matrizRecompensasAtras, V, factorDescuento)
        if error_maximo <= error:
            print(f"Deteniendo debido a que el error máximo {error_maximo} es menor o igual al umbral deseado {error}.")
            break
    return politica, V

In [20]:
error = 0.01
valores_ = iteracionPolitica(matrizAdelante, matrizRecompensasAdelante, matrizAtras, matrizRecompensasAtras, factorDescuento, epsilon, error)

Deteniendo debido a que el error máximo 0.0023547249801887937 es menor o igual al umbral deseado 0.01.


In [63]:
print(valores_[0]) 

[[0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0]]


In [78]:
def error_test(valores_optimos, indicador, matrizAdelante, matrizRecompensasAdelante, matrizAtras, matrizRecompensasAtras, factorDescuento, acciones):
    errno = [0] * len(matrizAdelante)
    bellman = bellman_cal(matrizAdelante, matrizRecompensasAdelante, matrizAtras, matrizRecompensasAtras, factorDescuento, acciones)
    for i in range(len(bellman)):
        stp = abs(bellman[i] - valores_optimos[i])
        if stp > 0 and stp <= indicador:
            errno[i] = 0
        else:
            errno[i] = 1
    ans = sum(errno)
    return ans

### **Hallar una política óptima para una instancia del MDP, usando iteración de valor.**

In [23]:
def iteraciondevalor(matrizAdelante, matrizRecompensasAdelante, matrizAtras, matrizRecompensasAtras, politica, factorDescuento, epsilon):
    valores = [0] * len(matrizAdelante)
    convergencia = False
    while not convergencia: 
        valores_previos = valores.copy()
        for estado in range(len(matrizAdelante)):
            acciones_valor = []
            valor_avanzar = sum(matrizAdelante[estado][estado_siguiente] * (matrizRecompensasAdelante[estado][estado_siguiente] + factorDescuento * valores[estado_siguiente]) for estado_siguiente in range(len(matrizAdelante[estado])))
            acciones_valor.append(valor_avanzar)
            valor_retroceder = sum(matrizAtras[estado][estado_siguiente] * (matrizRecompensasAtras[estado][estado_siguiente] + factorDescuento * valores[estado_siguiente]) for estado_siguiente in range(len(matrizAtras[estado])))
            acciones_valor.append(valor_retroceder)
            mejor_accion = acciones_valor.index(max(acciones_valor))
            valores[estado] = acciones_valor[mejor_accion]
        # Verificar si se ha alcanzado la convergencia    
        dif_maxima = max(abs(valores[estado] - valores_previos[estado]) for estado in range(len(matrizAdelante))) 
        if dif_maxima < epsilon:
            convergencia = True
    return valores

epsilon = 0.001
V = iteraciondevalor(matrizAdelante, matrizRecompensasAdelante, matrizAtras, matrizRecompensasAtras, politica, factorDescuento, epsilon)
# print("La política es: " + str(politica))
# print("El valor de descuento es: " + str(factorDescuento))
# print("Las probabilidades del dado son: " + str(p))
# print("Función de Valor de Estado:")
for i in range(len(V)):
    print("Casilla " +str(i+1)+" tiene un valor de "+ str(V[i]))

Casilla 1 tiene un valor de 0.6945656811999971
Casilla 2 tiene un valor de 0.7519849544693527
Casilla 3 tiene un valor de 0.7520509328032194
Casilla 4 tiene un valor de 0.752114266222114
Casilla 5 tiene un valor de 0.8391504842699508
Casilla 6 tiene un valor de 0.8392220746790685
Casilla 7 tiene un valor de 0.6943632590465554
Casilla 8 tiene un valor de 0.6943328957235392
Casilla 9 tiene un valor de 0.6224016427519489
Casilla 10 tiene un valor de 0.6029542492442582
Casilla 11 tiene un valor de 0.5805802466975799
Casilla 12 tiene un valor de 0.5417947110617243
Casilla 13 tiene un valor de 0.49718060651912255
Casilla 14 tiene un valor de 0.46760320864000765
Casilla 15 tiene un valor de 0.49687719973719624
Casilla 16 tiene un valor de 0.4780485332849833
Casilla 17 tiene un valor de 0.45931267589109215
Casilla 18 tiene un valor de 0.44112254027011893
Casilla 19 tiene un valor de 0.42602171465137817
Casilla 20 tiene un valor de 0.41534788087121643
Casilla 21 tiene un valor de 0.407509581705

## **Punto 3**

**Utilice su implementación de programación dinámica para hallar la función de valor de:**

**a) La política que siempre avanza.**

**b) La política que avanza con probabilidad 0,7.**


### **3.a. Hallar la función de valor para la política que siempre avanza**

A continuación se define la política que siempre avanza. Según la estructura que se está utilizando para definir la política, el primer valor debería ser 1 y el segundo 0. De esta forma, se estaría diciendo que con probabilidad 1 se escoge la opción de avanzar y con probabilidad 0 la opción de retroceder.

In [24]:
politica = []
for i in range(100):
    politica.append([1, 0])
V = calcularFuncionValorEstado(matrizAdelante, matrizRecompensasAdelante, matrizAtras, matrizRecompensasAtras, politica, factorDescuento, epsilon)
print("La política es: " +str(politica))
print("El valor de descuento es: " + str(factorDescuento))
print("Las probabilidades del dado son: " + str(p))
print("Función de Valor de Estado:")
for i in range(len(V)):
    print("Casilla " +str(i+1)+" tiene un valor de "+ str(V[i]))

La política es: [[1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0]]
El valor de descuento es: 0.9
Las probabilidades del dado son: [0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.166666666666

### **3.b. Hallar la función de valor para la política que avanza con probabilidad de 0.7**

A continuación se define la política que avanza con probabilidad de 0.7. Según la estructura que se está utilizando para definir la política, el primer valor debería ser 0.7 y el segundo 0.3. De esta forma, se estaría diciendo que con probabilidad 1 se escoge la opción de avanzar y con probabilidad 0 la opción de retroceder.

In [25]:
politica = []
for i in range(100):
    politica.append([0.7, 0.3])
V = calcularFuncionValorEstado(matrizAdelante, matrizRecompensasAdelante, matrizAtras, matrizRecompensasAtras, politica, factorDescuento, epsilon)
print("La política es: " +str(politica))
print("El valor de descuento es: " + str(factorDescuento))
print("Las probabilidades del dado son: " + str(p))
print("Función de Valor de Estado:")
for i in range(len(V)):
    print("Casilla " +str(i+1)+" tiene un valor de "+ str(V[i]))

La política es: [[0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3], [0.7, 0.3],

## **Punto 4**

**Utilice su implementación de iteración de política para encontrar una política óptima para la configuración mostrada en la figura para tres vectores p generados aleatoriamente. Chequee que la política encontrada en cada caso es efectivamente óptima. Muestre la política óptima en cada caso (acción en cada casilla). Repita el experimento pero ahora utilizando iteración de valor (usando los mismos p). Experimente en ambos casos con diferentes valores de γ.**

Se definen tres vectores p generados aleatoriamente:

In [ ]:
import numpy as np
np.random.seed(42)
p1 = np.random.dirichlet(np.ones(6), size=1)[0]
p2 = np.random.dirichlet(np.ones(6), size=1)[0]
p3 = np.random.dirichlet(np.ones(6), size=1)[0]
print("Vector de probabilidades p1:", p1)
print("Vector de probabilidades p2:", p2)
print("Vector de probabilidades p3:", p3)

Vector de probabilidades p1: [0.07758679 0.49768066 0.21770513 0.15094204 0.02804505 0.02804033]
Vector de probabilidades p2: [0.00772536 0.25965576 0.11865618 0.15895797 0.00268525 0.45231947]
Vector de probabilidades p3: [0.50534215 0.06751955 0.05676773 0.05731437 0.10261516 0.21044104]


Ahora se construyen los diferentes MDP para los diferentes vectores p y para los valores de γ ={1, 0.9, 0.8}

In [53]:
casillasAzules = [80, 100]    
casillasRojas = [23, 37, 45, 67, 89]
factorDescuento1 = 0.7
matrizAdelanteP1L1, matrizAtrasP1L1, matrizRecompensasAdelanteP1L1, matrizRecompensasAtrasP1L1 = construirMDP(casillasAzules, casillasRojas, p1, factorDescuento)
matrizAdelanteP2L1, matrizAtrasP2L1, matrizRecompensasAdelanteP2L1, matrizRecompensasAtrasP2L1 = construirMDP(casillasAzules, casillasRojas, p2, factorDescuento)
matrizAdelanteP3L1, matrizAtrasP3L1, matrizRecompensasAdelanteP3L1, matrizRecompensasAtrasP3L1 = construirMDP(casillasAzules, casillasRojas, p3, factorDescuento)

factorDescuento2 = 0.9
matrizAdelanteP1L09, matrizAtrasP1L09, matrizRecompensasAdelanteP1L09, matrizRecompensasAtrasP1L09 = construirMDP(casillasAzules, casillasRojas, p1, factorDescuento)
matrizAdelanteP2L09, matrizAtrasP2L09, matrizRecompensasAdelanteP2L09, matrizRecompensasAtrasP2L09 = construirMDP(casillasAzules, casillasRojas, p2, factorDescuento)
matrizAdelanteP3L09, matrizAtrasP3L09, matrizRecompensasAdelanteP3L09, matrizRecompensasAtrasP3L09 = construirMDP(casillasAzules, casillasRojas, p3, factorDescuento)

factorDescuento3 = 0.8
matrizAdelanteP1L08, matrizAtrasP1L08, matrizRecompensasAdelanteP1L08, matrizRecompensasAtrasP1L08 = construirMDP(casillasAzules, casillasRojas, p1, factorDescuento)
matrizAdelanteP2L08, matrizAtrasP2L08, matrizRecompensasAdelanteP2L08, matrizRecompensasAtrasP2L08 = construirMDP(casillasAzules, casillasRojas, p2, factorDescuento)
matrizAdelanteP3L08, matrizAtrasP3L08, matrizRecompensasAdelanteP3L08, matrizRecompensasAtrasP3L08 = construirMDP(casillasAzules, casillasRojas, p3, factorDescuento)    

### Utilizando la implementación de iteración de política

Cuando γ = 1, el valor presente de una serie de recompensas futuras constantes y positivas se vuelve infinito, lo que puede llevar a problemas de convergencia en los algoritmos de programación dinámica como la iteración de valor o la iteración de política. Un γ menor que 1 asegura que la suma de las recompensas descontadas futuras converja, lo que facilita la solución numérica del MDP.

In [60]:
V11 = iteracionPolitica(matrizAdelanteP1L1, matrizRecompensasAdelanteP1L1, matrizAtrasP1L1, matrizRecompensasAtrasP1L1, factorDescuento1, 0.001, 0.1)
V21 = iteracionPolitica(matrizAdelanteP2L1, matrizRecompensasAdelanteP2L1, matrizAtrasP2L1, matrizRecompensasAtrasP2L1, factorDescuento1, 0.001, 0.01)
V31 = iteracionPolitica(matrizAdelanteP3L1, matrizRecompensasAdelanteP3L1, matrizAtrasP3L1, matrizRecompensasAtrasP3L1, factorDescuento1, 0.001, 0.01)

V109 = iteracionPolitica(matrizAdelanteP1L09, matrizRecompensasAdelanteP1L09, matrizAtrasP1L09, matrizRecompensasAtrasP1L09, factorDescuento2, 0.001, 0.01)
V209 = iteracionPolitica(matrizAdelanteP2L09, matrizRecompensasAdelanteP2L09, matrizAtrasP2L09, matrizRecompensasAtrasP2L09, factorDescuento2, 0.001, 0.01)
V309 = iteracionPolitica(matrizAdelanteP3L09, matrizRecompensasAdelanteP3L09, matrizAtrasP3L09, matrizRecompensasAtrasP3L09, factorDescuento2, 0.001, 0.01)

V108 = iteracionPolitica(matrizAdelanteP1L08, matrizRecompensasAdelanteP1L08, matrizAtrasP1L08, matrizRecompensasAtrasP1L08, factorDescuento3, 0.001, 0.01)
V208 = iteracionPolitica(matrizAdelanteP2L08, matrizRecompensasAdelanteP2L08, matrizAtrasP2L08, matrizRecompensasAtrasP2L08, factorDescuento3, 0.001, 0.01)
V308 = iteracionPolitica(matrizAdelanteP3L08, matrizRecompensasAdelanteP3L08, matrizAtrasP3L08, matrizRecompensasAtrasP3L08, factorDescuento3, 0.001, 0.01)

Deteniendo debido a que el error máximo 0.02714120012993354 es menor o igual al umbral deseado 0.1.
Deteniendo debido a que el error máximo 0.0 es menor o igual al umbral deseado 0.01.
Deteniendo debido a que el error máximo 0.004347846889701912 es menor o igual al umbral deseado 0.01.
Deteniendo debido a que el error máximo 0.002936459185845175 es menor o igual al umbral deseado 0.01.
Deteniendo debido a que el error máximo 0.0 es menor o igual al umbral deseado 0.01.
Deteniendo debido a que el error máximo 0.0 es menor o igual al umbral deseado 0.01.
Deteniendo debido a que el error máximo 0.0 es menor o igual al umbral deseado 0.01.
Deteniendo debido a que el error máximo 0.0 es menor o igual al umbral deseado 0.01.
Deteniendo debido a que el error máximo 0.0 es menor o igual al umbral deseado 0.01.


A continuación se verifica que la política encontrada es efectivamente óptima:

In [89]:
error_1 = error_test(V11[1], 0.2, matrizAdelanteP1L1, matrizRecompensasAdelanteP1L1, matrizAtrasP1L1, matrizRecompensasAtrasP1L1, factorDescuento1, 2)
error_2 = error_test(V21[1], 0.2, matrizAdelanteP2L1, matrizRecompensasAdelanteP2L1, matrizAtrasP2L1, matrizRecompensasAtrasP2L1, factorDescuento1, 2)
error_3 = error_test(V31[1], 0.2, matrizAdelanteP3L1, matrizRecompensasAdelanteP3L1, matrizAtrasP3L1, matrizRecompensasAtrasP3L1, factorDescuento1, 2)

error_4 = error_test(V109[1], 0.2, matrizAdelanteP1L09, matrizRecompensasAdelanteP1L09, matrizAtrasP1L09, matrizRecompensasAtrasP1L09, factorDescuento2, 2)
error_5 = error_test(V209[1], 0.2, matrizAdelanteP2L09, matrizRecompensasAdelanteP2L09, matrizAtrasP2L09, matrizRecompensasAtrasP2L09, factorDescuento2, 2)
error_6 = error_test(V309[1], 0.2, matrizAdelanteP3L09, matrizRecompensasAdelanteP3L09, matrizAtrasP3L09, matrizRecompensasAtrasP3L09, factorDescuento2, 2)

error_7 = error_test(V108[1], 0.2, matrizAdelanteP1L08, matrizRecompensasAdelanteP1L08, matrizAtrasP1L08, matrizRecompensasAtrasP1L08, factorDescuento3, 2)
error_8 = error_test(V208[1], 0.2, matrizAdelanteP2L08, matrizRecompensasAdelanteP2L08, matrizAtrasP2L08, matrizRecompensasAtrasP2L08, factorDescuento3, 2)
error_9 = error_test(V308[1], 0.2, matrizAdelanteP3L08, matrizRecompensasAdelanteP3L08, matrizAtrasP3L08, matrizRecompensasAtrasP3L08, factorDescuento3, 2)

In [90]:
for i in range(9):
    print(locals()[f"error_{i+1}"])

24
10
20
100
100
100
73
39
54


A continuación se muestra la política encontrada:

In [93]:
print(V11[0])

[[1, 0], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [0, 1], [1, 0], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0]]


### Utilizando la implementación de iteración de valor

In [ ]:
epsilon = 0.001
V = iteracionValor(matrizAdelante, matrizRecompensasAdelante, matrizAtras, matrizRecompensasAtras, politica, factorDescuento, epsilon)
print("La política es: " +str(politica))
print("El valor de descuento es: " + str(factorDescuento))
print("Las probabilidades del dado son: " + str(p))
print("Función de Valor de Estado:")
for i in range(len(V)):
    print("Casilla " +str(i+1)+" tiene un valor de "+ str(V[i]))

In [ ]:
verificarPolitica()

A continuación se muestra la política encontrada:

In [ ]:
mostrarPolitica()

## **Conclusiones**